In [1]:
import numpy as np
import tensorflow as tf
import tensorflow_graphics as tfg
from tensorflow_graphics.nn.loss import chamfer_distance
from vedo import *
from ipyvtklink.viewer import ViewInteractiveWidget
from matplotlib import pyplot as p
from scipy.spatial.transform import Rotation as R
from lidar_nerf_utils import *

import torch
from chamferdist import ChamferDistance

# %load_ext autoreload
# %autoreload 2
# %autosave 180

2024-10-09 16:10:16.988802: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-09 16:10:17.086334: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-09 16:10:17.405694: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/lib:/usr/local/lib:
2024-10-09 16:10:17.405757: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7:

In [2]:
#used to train NeRF
images = np.load("/home/derm/Desktop/imagesUndistortedV7.npy")
poses = np.load("/home/derm/Desktop/posesUndistortedV7.npy")
rays_o_all = np.load("/home/derm/Desktop/rays_oUndistortedV7.npy")
rays_d_all = np.load("/home/derm/Desktop/rays_dUndistortedV7.npy")

In [3]:
#Newer College Data
dir_name = "/media/derm/06EF-127D4/Newer College Dataset/"
experiment_name = "01_short_experiment-20230331T172433Z-009/01_short_experiment/"
fn_gt = dir_name + experiment_name + "ground_truth/registered_poses.csv"
#sec,nsec,x,y,z,qx,qy,qz,qw
gt = np.loadtxt(fn_gt, delimiter=',',skiprows = 1)
seconds = gt[:, 0]
nano_seconds = gt[:, 1]
xyz = gt[:, 2:5]
qxyzw = gt[:, 5:]
num_poses = qxyzw.shape[0]
sensor_poses = np.eye(4, dtype=np.float64).reshape(1, 4, 4).repeat(num_poses, axis=0)
sensor_poses[:, :3, :3] = R.from_quat(qxyzw).as_matrix()
sensor_poses[:, :3, 3] = xyz
T_CL = np.eye(4, dtype=np.float32)
T_CL[:3, :3] = R.from_quat([0.0, 0.0, 0.924, 0.383]).as_matrix()
T_CL[:3, 3] = np.array([-0.084, -0.025, 0.050], dtype=np.float32)
sensor_poses = np.einsum("nij,jk->nik", sensor_poses, T_CL)
initial_pose = np.linalg.inv(sensor_poses[0]) 
poses_timestamps = seconds * 10e9 + nano_seconds
sensor_poses = np.einsum("ij,njk->nik", np.linalg.inv(sensor_poses[0]), sensor_poses) #TRY COMMENTING OUT...

#get body frame vel to remove motion disortion from training data
vel_world_frame = np.diff(sensor_poses[:,:3,-1], axis = 0)
vel_body_frame = np.linalg.pinv(sensor_poses[1:,:3,:3]) @ vel_world_frame[:,:,None]
vel_body_frame = vel_body_frame[:,:,0]
#smooth out velocity estimates
def moving_average(a, n=10):
    ret = np.cumsum(a, dtype=float)
    ret[n:] = ret[n:] - ret[:-n]
    return ret[n - 1:] / n
window=50
MAx = moving_average(vel_body_frame[:,0], n = window)
MAy = moving_average(vel_body_frame[:,1], n = window)
MAz = moving_average(vel_body_frame[:,2], n = window)
vel_body_frame = np.array([MAx, MAy, MAz]).T

rot_vel_euls = np.diff(R.from_matrix(sensor_poses[:,:3,:3]).as_euler('xyz'), axis = 0)
idx = np.argwhere(rot_vel_euls > (np.pi))
rot_vel_euls[idx] = 0
idx = np.argwhere(rot_vel_euls < (-np.pi))
rot_vel_euls[idx] = 0

pl = '/media/derm/06EF-127D4/Newer College Dataset/new-college-29-01-2020-1cm-resolution-1stSection - mesh.ply'
HD_map = trimesh.load(pl).vertices
show_nth = 5 #10
submap = HD_map[::show_nth]

### Aligning raw scans with dataset-provided HD Map (instead of the one I stitched together). Try calculating performance metrics excluding points in nerf output that fall outside convex hull of provided HD Map.

This is a bad way to evaluate performance since it throws out points in interior rooms that are excluded from the provided Newer College HD map (we should do especially well there)

In [36]:
plt = Plotter(N = 1, axes = 0, bg = (1, 1, 1), interactive = True) #axes = 4 (simple), 1(scale)
disp=[]

frame_idx = 7650 + 480
start_frame_idx = 480
nerf_pc = np.load("generatedPointClouds/" + str(frame_idx) + "NeRF.npy") 
real_pc = np.load("generatedPointClouds/" + str(frame_idx) + "real.npy")

#remove too close
d = np.sqrt(np.sum(nerf_pc**2, axis = 1))
good  = np.argwhere(d > 2.)[:,0]
nerf_pc = nerf_pc[good]
#bring to map frame
nerf_to_map = sensor_poses[frame_idx].copy()
nerf_pc = (np.linalg.pinv(initial_pose) @ nerf_to_map @ np.append(nerf_pc, np.ones([len(nerf_pc), 1]), axis = 1).T).T
nerf_pc = nerf_pc[:,:3]

#throw out points outside convex hull
hull = ConvexHull(submap)
deln = Delaunay(submap)
inside_mask = deln.find_simplex(nerf_pc) > 0
nerf_pc = nerf_pc[inside_mask]

hull_points = submap[hull.vertices]
convex_mesh = vedo.ConvexHull(hull_points)
convex_mesh.alpha(0.1)
disp.append(convex_mesh)

# real_pc = (np.linalg.pinv(initial_pose) @ nerf_to_map @ np.append(real_pc, np.ones([len(real_pc), 1]), axis = 1).T).T
# real_pc = real_pc[:,:3]
# disp.append(Points(real_pc[:,:], c = "red", r = 3., alpha = 0.9))

disp.append(Points(nerf_pc[:,:], c = "blue", r = 3., alpha = 0.9))
disp.append(Points(submap[:,:3], c = "gray", r = 2, alpha = 0.5))
plt.show(disp, "aligning nerf output and real scans with provided HD map")
ViewInteractiveWidget(plt.window)

ViewInteractiveWidget(height=960, layout=Layout(height='auto', width='100%'), width=960)

In [41]:
#recenter at origin to crop

test = (np.linalg.pinv(nerf_to_map) @ initial_pose @ np.append(nerf_pc, np.ones([len(nerf_pc), 1]), axis = 1).T).T
test = test[:,:3]

map_nerfframe = (np.linalg.pinv(nerf_to_map) @ initial_pose @ np.append(submap, np.ones([len(submap), 1]), axis = 1).T).T
map_nerfframe = map_nerfframe[:,:3]

plt = Plotter(N = 1, axes = 0, bg = (1, 1, 1), interactive = True) #axes = 4 (simple), 1(scale)
disp=[]

disp.append(Points(test, c = "blue", r = 3., alpha = 0.9))
disp.append(Points(map_nerfframe, c = "gray", r = 3., alpha = 0.9))
plt.show(disp, "test")
ViewInteractiveWidget(plt.window)

ViewInteractiveWidget(height=960, layout=Layout(height='auto', width='100%'), width=960)

In [32]:
# try throwing out points in cloud A that don't fall within convex hull of cloud B
from scipy.spatial import ConvexHull, Delaunay
import vedo

a = np.random.randn(100,3)
b = np.random.randn(100,3) * np.array([2.0, 1.0, 0.5])

hull = ConvexHull(a)
deln = Delaunay(a)
inside_mask = deln.find_simplex(b) > 0

c = b[inside_mask]

plt = Plotter(N = 1, axes = 0, bg = (1, 1, 1), interactive = True) #axes = 4 (simple), 1(scale)
disp=[]
disp.append(Points(a, c = "red", r = 5., alpha = 0.5))
disp.append(Points(b, c = "blue", r =5., alpha = 0.5))
disp.append(Points(c, c = "purple", r =10., alpha = 0.3))

hull_points = a[hull.vertices]  # Points that form the vertices of the hull
convex_mesh = vedo.ConvexHull(hull_points)
convex_mesh.alpha(0.1)
disp.append(convex_mesh)

plt.show(disp, "convex hull test")
ViewInteractiveWidget(plt.window)

ViewInteractiveWidget(height=960, layout=Layout(height='auto', width='100%'), width=960)

### Newer College --- Loop through scans generated by NeRF, align with stitched HD Map, crop, and calculate performance metrics on each slice individually

In [4]:
#DEBUG-- figure out how to crop nerf

# for frame_idx in range(7650,8370,120): #7650,8850
frame_idx = 8370
crop_slices = 8
print("Frame: ", frame_idx)
start_frame_idx = frame_idx - 7650 #needed to index poses (data generation starts a 7650)

full_nerf = np.zeros([0,3])
full_real = np.zeros([0,3])

#will run out of memory if we try to calculate forward and reverse distances for entire HD map
# for c in range(crop_slices):
for crop_part in range(6):
    if crop_part > 2:
        c = crop_part +2
    else:
        c = crop_part
    #TODO-- researcher is constantly blocking sensor at ""  o'clock position
    #need to not look at these slices

    nerf_pc = np.load("generatedPointClouds/" + str(frame_idx) + "NeRF.npy") 
    real_pc = np.load("generatedPointClouds/" + str(frame_idx) + "real.npy")

    # #remove points too close to origin
    d = np.sqrt(np.sum(nerf_pc**2, axis = 1))
    good  = np.argwhere(d > 2.)[:,0]
    nerf_pc = nerf_pc[good]
    d = np.sqrt(np.sum(real_pc**2, axis = 1))
    good  = np.argwhere(d > 2.)[:,0]
    real_pc = real_pc[good]

    minb = -np.pi + (c/crop_slices)*2*np.pi
    maxb = -np.pi + ((c+1)/crop_slices)*2*np.pi

    #crop NeRF scan to azimuthal window~~~~~~~~~~~~~~
    nerf_pc_spherical = cartesian_to_spherical(nerf_pc)
    order = np.argsort(nerf_pc_spherical[:,1])#sort by azimuth angle
    nerf_pc_spherical = nerf_pc_spherical.numpy()[order,:]

    #crop by azimuth angle
    nerf_pc_spherical = nerf_pc_spherical[nerf_pc_spherical[:,1] < maxb]
    nerf_pc_spherical = nerf_pc_spherical[nerf_pc_spherical[:,1] > minb]
    nerf_pc = spherical_to_cartesian(nerf_pc_spherical).numpy()
    
    full_nerf = np.append(full_nerf, nerf_pc, axis =0)
    
    #crop raw scan to azimuthal window~~~~~~~~~~~~~~~
    real_pc_spherical = cartesian_to_spherical(real_pc)
    order = np.argsort(real_pc_spherical[:,1])#sort by azimuth angle
    real_pc_spherical = real_pc_spherical.numpy()[order,:]

    #crop by azimuth angle
    real_pc_spherical = real_pc_spherical[real_pc_spherical[:,1] < maxb]
    real_pc_spherical = real_pc_spherical[real_pc_spherical[:,1] > minb]
    real_pc = spherical_to_cartesian(real_pc_spherical).numpy()
    full_real = np.append(full_real, real_pc, axis =0)
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

        
plt = Plotter(N = 1, axes = 0, bg = (1, 1, 1), interactive = True) #axes = 4 (simple), 1(scale)
disp=[]

# disp.append(Points(real_pc[:,:], c = "red", r = 3., alpha = 0.9))
disp.append(Points(full_real[:,:], c = "red", r = 3., alpha = 0.9))
disp.append(Points(full_nerf[:,:], c = "blue", r = 3., alpha = 0.9))

plt.show(disp, "testing CD")
ViewInteractiveWidget(plt.window)

Frame:  8370


ViewInteractiveWidget(height=960, layout=Layout(height='auto', width='100%'), width=960)

In [ ]:
#map created by stitching together undistorted raw lidar scans 
hd_map = np.load("generatedPointClouds/stitched_map.npy")
crop_slices = 8

cd_hist = []
accuracy_hist = []
completeness_hist = []
good1_hist = []
good2_hist = []

for frame_idx in range(7650,7770,30): #7650,8850
    print("Frame: ", frame_idx)
    start_frame_idx = frame_idx - 7650 #needed to index poses (data generation starts a 7650)

    #will run out of memory if we try to calculate forward and reverse distances for entire HD map
#     for c in range(crop_slices):
    for crop_part in range(6):
        #researcher is constantly blocking sensor at "~2-4"  o'clock position
        #need to not look at this part of each scan to keep things fair
        if crop_part > 2:
            c = crop_part +2
        else:
            c = crop_part

        nerf_pc = np.load("generatedPointClouds/" + str(frame_idx) + "NeRF.npy") 
        real_pc = np.load("generatedPointClouds/" + str(frame_idx) + "real.npy")

        # #remove points too close to origin
        d = np.sqrt(np.sum(nerf_pc**2, axis = 1))
        good  = np.argwhere(d > 2.)[:,0]
        nerf_pc = nerf_pc[good]
        d = np.sqrt(np.sum(real_pc**2, axis = 1))
        good  = np.argwhere(d > 2.)[:,0]
        real_pc = real_pc[good]
    
        minb = -np.pi + (c/crop_slices)*2*np.pi
        maxb = -np.pi + ((c+1)/crop_slices)*2*np.pi

        #crop NeRF scan to azimuthal window~~~~~~~~~~~~~~
        nerf_pc_spherical = cartesian_to_spherical(nerf_pc)
        order = np.argsort(nerf_pc_spherical[:,1])#sort by azimuth angle
        nerf_pc_spherical = nerf_pc_spherical.numpy()[order,:]

        #crop by azimuth angle
        nerf_pc_spherical = nerf_pc_spherical[nerf_pc_spherical[:,1] < maxb]
        nerf_pc_spherical = nerf_pc_spherical[nerf_pc_spherical[:,1] > minb]
        nerf_pc = spherical_to_cartesian(nerf_pc_spherical).numpy()

        #crop raw scan to azimuthal window~~~~~~~~~~~~~~~
        real_pc_spherical = cartesian_to_spherical(real_pc)
        order = np.argsort(real_pc_spherical[:,1])#sort by azimuth angle
        real_pc_spherical = real_pc_spherical.numpy()[order,:]

        #crop by azimuth angle
        real_pc_spherical = real_pc_spherical[real_pc_spherical[:,1] < maxb]    
        real_pc_spherical = real_pc_spherical[real_pc_spherical[:,1] > minb]    
        real_pc = spherical_to_cartesian(real_pc_spherical).numpy()

        # center map on NeRF scan so we can crop it by azimuth angle to prevent OOM errors ~~~~~~~~~~~~
        n_rots = 128
        n_vert_patches = 1
        n_cols_to_skip = n_rots // 8 
        poses_index = start_frame_idx*(n_rots * n_vert_patches - 2*n_cols_to_skip*n_vert_patches)
        nerf_to_map = poses[poses_index].copy()
        nerf_to_map[:3,-1] *= 200
        map_nerfframe = (np.linalg.pinv(nerf_to_map) @ np.append(hd_map, np.ones([len(hd_map), 1]), axis = 1).T).T
        map_nerfframe = map_nerfframe[:,:3] @ R.from_euler('xyz', [0.,0., np.pi/4 - np.pi/128]).as_matrix().T

        #crop HD Map to azimuthal window
        map_nerfframe_spherical = cartesian_to_spherical(map_nerfframe)
        order = np.argsort(map_nerfframe_spherical[:,1])#sort by azimuth angle
        map_nerfframe_spherical = map_nerfframe_spherical.numpy()[order,:]
        #crop by azimuth angle
        map_nerfframe_spherical = map_nerfframe_spherical[map_nerfframe_spherical[:,1] < maxb] #    
        map_nerfframe_spherical = map_nerfframe_spherical[map_nerfframe_spherical[:,1] > minb] #    

        map_nerfframe = spherical_to_cartesian(map_nerfframe_spherical).numpy()
        #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
        try:
            #calculate accuracy and completeness
            nerf_pc_tensor = torch.tensor(nerf_pc.copy(), dtype=torch.float32)
            map_pc_tensor = torch.tensor(map_nerfframe[:,:3].copy(), dtype=torch.float32)
            real_pc_tensor = torch.tensor(real_pc.copy(), dtype=torch.float32)

            #Accuracy--
            #"The accuracy metric is defined as the mean distance of points on the output 
            #   mesh to their nearest neighbors on the ground truth mesh."
            ## Like SHINE mapping, for the "forward" distance we comare nerf-generated PCs against an 
            ##  HD-Map produced by stitching together undistorted raw scans (since no surfaces outside the courtyard
            ##  are provided in the NC HD Map)
            #map is used as to not overly punish highly oblique surfaces (none of these methods explicitly model beam spreading) 
            dists1 = torch.cdist(nerf_pc_tensor, map_pc_tensor)
            min_dists1 = dists1.min(dim=1)[0]  # from nerf to stitched map
            accuracy = min_dists1.mean()

            #completeness--
            dists2 = torch.cdist(nerf_pc_tensor, real_pc_tensor)
            min_dists2 = dists2.min(dim=0)[0]  # from real_pc to nerf_output
            completeness = min_dists2.mean()

            #chamfer distance
            cd = (min_dists1.mean() + min_dists2.mean())/2

            cd_hist = np.append(cd_hist, cd)
            accuracy_hist = np.append(accuracy_hist, accuracy)
            completeness_hist = np.append(completeness_hist, completeness)

            #get fraction under distance threshold for calculating F-score
            good1 = np.where(min_dists1 < 0.2)[0]
            good1 = len(good1)/len(min_dists1)
            good2 = np.where(min_dists2 < 0.2)[0]
            good2 = len(good2)/len(min_dists2)
            good1_hist = np.append(good1_hist, good1)
            good2_hist = np.append(good2_hist, good2)

            print("\n cd:", cd, "  accuracy: ", accuracy, " completeness:", completeness)
        except:
            print("error with numerics c = ", c)
        print("good1, good2:", good1, good2)
        #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    
plt = Plotter(N = 1, axes = 0, bg = (1, 1, 1), interactive = True) #axes = 4 (simple), 1(scale)
disp=[]

print("real_pc:", len(real_pc))
print("nerf_pc:", len(nerf_pc))
print("map_nerfframe:", len(map_nerfframe))

disp.append(Points(real_pc[:,:], c = "red", r = 3., alpha = 0.9))
disp.append(Points(nerf_pc[:,:], c = "blue", r = 3., alpha = 0.9))
disp.append(Points(map_nerfframe[:,:3], c = "gray", r = 2, alpha = 0.5))

plt.show(disp, "testing CD")
ViewInteractiveWidget(plt.window)

In [ ]:
print(np.mean(cd_hist))
print(np.mean(accuracy_hist))
print(np.mean(completeness_hist))

# print(np.mean(good1_hist))
# print(np.mean(good2_hist))
print(2*(np.mean(good1_hist)*np.mean(good2_hist))/(np.mean(good1_hist) + np.mean(good2_hist)))

# fig, ax = p.subplots()
# ax.plot(cd_hist)

In [ ]:
# nerf_pc = np.load("lidar_nerf_demo/generated_pc_at_start.npy")
# real_pc = np.load("lidar_nerf_demo/real_pc_at_start.npy")
# nerf_pc = np.random.randn(1000,3)
# real_pc = np.random.randn(1000,3)
# nerf_pc = np.load("generatedPointClouds/7650NeRF.npy")
# real_pc = np.load("generatedPointClouds/7650real.npy")
# nerf_pc = np.load("generatedPointClouds/7750NeRF.npy")
# real_pc = np.load("generatedPointClouds/7750real.npy")
nerf_pc = np.load("generatedPointClouds/8130NeRF.npy")
real_pc = np.load("generatedPointClouds/8130real.npy")
# nerf_pc = np.load("generatedPointClouds/MaiCityNeRFOutput.npy")
# real_pc = np.load("generatedPointClouds/MaiCityReal.npy")

##use "HD map" comprised of undistorted raw scans stitched together
## (true HD map provided with dataset it cropped to not include interior features) 
hd_map = np.load("generatedPointClouds/stitched_map.npy")
# real_pc = real_pc[::20]

#train set
# nerf_pc = np.load("generatedPointClouds/MaiCityNeRFOutput_frame_10.npy") 
# real_pc = np.load("generatedPointClouds/MaiCityReal_frame_10.npy")
# # #test set
# nerf_pc = np.load("generatedPointClouds/MaiCityNeRFOutput_v2_frame_0.npy") 
# real_pc = np.load("generatedPointClouds/MaiCityReal_v2_frame_0.npy")

# #remove ground plane
# real_pc = real_pc[real_pc[:,2] > -1]
# nerf_pc = nerf_pc[nerf_pc[:,2] > -1]

#crop real PC down the same as NeRF output so bidirectional CD doesn't punish non-overlaps
# real_pc = real_pc[len(real_pc)//8:7*len(real_pc)//8,:]
# real_pc = real_pc[len(real_pc)//8:31*len(real_pc)//32,:]


# #remove points too close to origin
d = np.sqrt(np.sum(nerf_pc**2, axis = 1))
good  = np.argwhere(d > 2.)[:,0]
nerf_pc = nerf_pc[good]


#for NC only
# nerf_pc = np.flip(nerf_pc, axis = 0)

# #remove points out of range of sensor (only needed for Mai City)
# d = np.sqrt(np.sum(nerf_pc**2, axis = 1))
# good = np.argwhere(d < 100)[:,0]
# nerf_pc = nerf_pc[good]


# spin_part = 1
# splits = 4 #8
# nerf_pc = nerf_pc[spin_part*len(nerf_pc)//splits:(spin_part+1)*len(nerf_pc)//splits,:]
# # real_pc = real_pc[spin_part*len(real_pc)//splits:(spin_part+1)*len(real_pc)//splits,:]


# #crop NeRF scan to azimuthal window~~~~~~~~~~~~~~
# nerf_pc_spherical = cartesian_to_spherical(nerf_pc)
# order = np.argsort(nerf_pc_spherical[:,1])#sort by azimuth angle
# nerf_pc_spherical = nerf_pc_spherical.numpy()[order,:]

# #crop by azimuth angle
# nerf_pc_spherical = nerf_pc_spherical[nerf_pc_spherical[:,1] < -3*np.pi/4]

# nerf_pc = spherical_to_cartesian(nerf_pc_spherical).numpy()
# print(len(nerf_pc))
# #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

#bring nerf generated pc to map frame ~~~~~~~~~~~~~
start_frame_idx = 480 #120
n_rots = 128
n_vert_patches = 1
n_cols_to_skip = n_rots // 8 
poses_index = start_frame_idx*(n_rots * n_vert_patches - 2*n_cols_to_skip*n_vert_patches)
# print(poses[poses_index])
nerf_to_map = poses[poses_index].copy()
nerf_to_map[:3,-1] *= 200
# nerf_to_map[:3,:3] = nerf_to_map[:3,:3] @ R.from_euler('xyz', [0.,0.,-np.pi/4]).as_matrix()

nerf_pc_rotated = nerf_pc @ R.from_euler('xyz', [0.,0.,np.pi/4 - np.pi/128]).as_matrix()
nerf_mapframe = (nerf_to_map @ np.append(nerf_pc_rotated, np.ones([len(nerf_pc), 1]), axis = 1).T).T
nerf_mapframe = nerf_mapframe[:,:3]
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

# # center map on NeRF scan so we can crop it to prevent the dreaded OOM ~~~~~~~~
# map_nerfframe = (np.linalg.pinv(nerf_to_map) @ np.append(real_pc, np.ones([len(map_rotated), 1]), axis = 1).T).T
# map_nerfframe = map_nerfframe[:,:3] @ R.from_euler('xyz', [0.,0., np.pi/4 - np.pi/128]).as_matrix().T

# #crop HD Map to azimuthal window
# map_nerfframe_spherical = cartesian_to_spherical(map_nerfframe)
# order = np.argsort(map_nerfframe_spherical[:,1])#sort by azimuth angle
# map_nerfframe_spherical = map_nerfframe_spherical.numpy()[order,:]

# #crop by azimuth angle
# map_nerfframe_spherical = map_nerfframe_spherical[map_nerfframe_spherical[:,1] < -3*np.pi/4] #debug for viz

# map_nerfframe = spherical_to_cartesian(map_nerfframe_spherical).numpy()
# print(len(map_nerfframe))
# #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~


print(np.shape(nerf_pc))
print(np.shape(real_pc))

In [ ]:
#combine multiple nerf-generated point clouds to new HD Map for visualization

nerf_map = np.zeros([0,3])

for frame_idx in range(7650,8850,120): #7650,8850,30
    print("Frame: ", frame_idx)
    start_frame_idx = frame_idx - 7650 #needed to index poses (data generation starts a 7650)

#     nerf_pc = np.load("generatedPointClouds/" + str(frame_idx) + "NeRF.npy") 
    nerf_pc = np.load("generatedPointClouds/" + str(frame_idx) + "NeRF_v2.npy") #override roll = 0.0001 on render 
    # #remove points too close to origin
    d = np.sqrt(np.sum(nerf_pc**2, axis = 1))
    good  = np.argwhere(d > 2.)[:,0]
    nerf_pc = nerf_pc[good]

    n_rots = 128
    n_vert_patches = 1
    n_cols_to_skip = n_rots // 8 
    poses_index = start_frame_idx*(n_rots * n_vert_patches - 2*n_cols_to_skip*n_vert_patches)
    # print(poses[poses_index])
    nerf_to_map = poses[poses_index].copy()
    nerf_to_map[:3,-1] *= 200
    # nerf_to_map[:3,:3] = nerf_to_map[:3,:3] @ R.from_euler('xyz', [0.,0.,-np.pi/4]).as_matrix()

    nerf_pc_rotated = nerf_pc @ R.from_euler('xyz', [0.,0.,np.pi/4 - np.pi/128]).as_matrix()
    nerf_mapframe = (nerf_to_map @ np.append(nerf_pc_rotated, np.ones([len(nerf_pc), 1]), axis = 1).T).T
    nerf_mapframe = nerf_mapframe[:,:3]
    
    nerf_map = np.append(nerf_map, nerf_mapframe, axis = 0)

    
#downsample using voxel grid
import open3d as o3d
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(nerf_map)
voxel_size = 0.02
pcd = pcd.voxel_down_sample(voxel_size)
nerf_map = np.asarray(pcd.points)


#crop
nerf_map = nerf_map[nerf_map[:,1] < 50]
nerf_map = nerf_map[nerf_map[:,0] < 25]
print(len(nerf_map))

# save to .ply file to render with open3d
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(nerf_map)
pcd.estimate_normals()
o3d.io.write_point_cloud("newerCollegePLINKMap.ply", pcd)
np.save("generatedPointClouds/newerCollegePLINKMap.npy", nerf_map)

plt = Plotter(N = 1, axes = 0, bg = (1, 1, 1), interactive = True) #axes = 4 (simple), 1(scale)
disp=[]

disp.append(Points(nerf_map, c = "gray", r = 3., alpha = 0.1))
# disp.append(Points(hd_map[:,:3], c = "gray", r = 2, alpha = 0.5))

plt.show(disp, "Combining NeRF-Generated Scans")
ViewInteractiveWidget(plt.window)

In [ ]:
maxb = -np.pi/4
minb = -np.pi

#crop NeRF scan to azimuthal window~~~~~~~~~~~~~~
nerf_pc_spherical = cartesian_to_spherical(nerf_pc)
order = np.argsort(nerf_pc_spherical[:,1])#sort by azimuth angle
nerf_pc_spherical = nerf_pc_spherical.numpy()[order,:]

#crop by azimuth angle
nerf_pc_spherical = nerf_pc_spherical[nerf_pc_spherical[:,1] < maxb]
nerf_pc_spherical = nerf_pc_spherical[nerf_pc_spherical[:,1] > minb]
nerf_pc = spherical_to_cartesian(nerf_pc_spherical).numpy()

#crop raw scan to azimuthal window~~~~~~~~~~~~~~~
real_pc_spherical = cartesian_to_spherical(real_pc)
order = np.argsort(real_pc_spherical[:,1])#sort by azimuth angle
real_pc_spherical = real_pc_spherical.numpy()[order,:]

#crop by azimuth angle
real_pc_spherical = real_pc_spherical[real_pc_spherical[:,1] < maxb]    
real_pc_spherical = real_pc_spherical[real_pc_spherical[:,1] > minb]    
real_pc = spherical_to_cartesian(real_pc_spherical).numpy()


# center map on NeRF scan so we can crop it to prevent the dreaded OOM ~~~~~~~~
start_frame_idx = 480 #120
n_rots = 128
n_vert_patches = 1
n_cols_to_skip = n_rots // 8 
poses_index = start_frame_idx*(n_rots * n_vert_patches - 2*n_cols_to_skip*n_vert_patches)
# print(poses[poses_index])
nerf_to_map = poses[poses_index].copy()
nerf_to_map[:3,-1] *= 200
map_nerfframe = (np.linalg.pinv(nerf_to_map) @ np.append(hd_map, np.ones([len(hd_map), 1]), axis = 1).T).T
map_nerfframe = map_nerfframe[:,:3] @ R.from_euler('xyz', [0.,0., np.pi/4 - np.pi/128]).as_matrix().T

#crop HD Map to azimuthal window
map_nerfframe_spherical = cartesian_to_spherical(map_nerfframe)
order = np.argsort(map_nerfframe_spherical[:,1])#sort by azimuth angle
map_nerfframe_spherical = map_nerfframe_spherical.numpy()[order,:]

#crop by azimuth angle
# map_nerfframe_spherical = map_nerfframe_spherical[map_nerfframe_spherical[:,1] < -np.pi/2] #debug for viz

map_nerfframe = spherical_to_cartesian(map_nerfframe_spherical).numpy()
print(len(map_nerfframe))
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

plt = Plotter(N = 1, axes = 0, bg = (1, 1, 1), interactive = True) #axes = 4 (simple), 1(scale)
disp=[]

disp.append(Points(real_pc[:,:], c = "red", r = 3., alpha = 0.9))
disp.append(Points(nerf_pc[:,:], c = "blue", r = 3., alpha = 0.9))
disp.append(Points(map_nerfframe[:,:3], c = "gray", r = 2, alpha = 0.5))

plt.show(disp, "testing CD")
ViewInteractiveWidget(plt.window)

In [ ]:
points1 = torch.tensor(nerf_pc.copy(), dtype=torch.float32)
points2 = torch.tensor(map_nerfframe[:,:3].copy(), dtype=torch.float32)

dists = torch.cdist(points1, points2)
# print(dists)
# Find the minimum distances
min_dists1 = dists.min(dim=1)[0]  # from points1 to points2

#Accuracy--
#"The accuracy metric is defined as the mean distance of points on the output 
#   mesh to their nearest neighbors on the ground truth mesh." --
accuracy = min_dists1.mean()
print("accuracy:", accuracy)

In [ ]:
import open3d as o3d
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(map_nerfframe[:,:3])
o3d.io.write_point_cloud("NCStitchedMap.ply", pcd)

In [ ]:
# real_pc = np.linspace(0,9,10)[:,None]*np.array([1.,0.,0.])
# nerf_pc = np.linspace(0,9,10)[:,None]*np.array([1.,1.,0.]) + np.array([0.,1.,0])

plt = Plotter(N = 1, axes = 0, bg = (1, 1, 1), interactive = True) #axes = 4 (simple), 1(scale)
disp=[]

# disp.append(Points(real_pc[:,:], c = "red", r = 5))
disp.append(Points(real_pc[:,:], c = "gray", r = 2, alpha = 0.5))
# disp.append(Points(nerf_pc[:,:], c = "blue", r = 3., alpha = 0.9))
disp.append(Points(nerf_mapframe[:,:], c = "blue", r = 3., alpha = 0.9))

plt.show(disp, "testing CD")
ViewInteractiveWidget(plt.window)


# manually calcluate CD with torch

In [ ]:
points1 = torch.tensor(nerf_pc.copy(), dtype=torch.float32)
points2 = torch.tensor(real_pc.copy(), dtype=torch.float32)

dists = torch.cdist(points1, points2)
# print(dists)
# Find the minimum distances
min_dists1 = dists.min(dim=1)[0]  # from points1 to points2

#Accuracy--
#"The accuracy metric is defined as the mean distance of points on the output 
#   mesh to their nearest neighbors on the ground truth mesh." --
accuracy = min_dists1.mean()
print("accuracy:", accuracy)

min_dists2 = dists.min(dim=0)[0]  # from points2 to points1

#completeness--
# The completeness metric is defined similarly, but in opposite direction.
completeness = min_dists2.mean()
print("completeness:", completeness)

#chamfer distance
cd = (min_dists1.mean() + min_dists2.mean())/2
print("cd:", cd)


In [ ]:
good2 = np.where(min_dists2 < 0.2)[0]
good2 = len(good2)/len(min_dists2)

print(good2)

In [ ]:
# (15.59+6.86)/2

#exclude very beginning and end where points don't align due to staggering
test1 = min_dists1[1000:-1000]
test2 = min_dists2[1000:-1000]

thresh = np.sqrt((np.sqrt(2)*.2)**2 + .2**2 )
print(thresh)
test1 = test1[test1 < thresh]
test2 = test2[test2 < thresh]

print(test1.mean(), test2.mean())
print(test1.std(), test2.std())

fig, ax = p.subplots()

# ax.plot(min_dists1)
ax.plot(test1)
ax.set_xlim([4000,5000])

print(min_dists1)

In [ ]:
cd_hist = []
accuracy_hist = []
completeness_hist = []
good1_hist = []
good2_hist = []

for frame_idx in range(1): #25
    print("frame: ", frame_idx, "---------------")

    for spin_part in range(8):
#         nerf_pc = np.load("generatedPointClouds/MaiCityNeRFOutput_frame_" + str(frame_idx) + ".npy")
#         real_pc = np.load("generatedPointClouds/MaiCityReal_frame_" + str(frame_idx) + ".npy")
        nerf_pc = np.load("generatedPointClouds/MaiCityNeRFOutput_v2_frame_" + str(frame_idx) + ".npy")
        real_pc = np.load("generatedPointClouds/MaiCityReal_v2_frame_" + str(frame_idx) + ".npy")

        # #remove ground plane
        # real_pc = real_pc[real_pc[:,2] > -1]
        # nerf_pc = nerf_pc[nerf_pc[:,2] > -1]

        # #debug: create simple pc
        # nerf_pc = np.array([[0,0,0], 
        #                     [1,1,1]])
        # real_pc = np.array([[0,0,0], 
        #                     [0,0,0]])

        #crop real PC down the same as NeRF output so bidirectional CD doesn't punish non-overlaps
        # real_pc = real_pc[len(real_pc)//8:7*len(real_pc)//8,:]
        # real_pc = real_pc[len(real_pc)//8:29*len(real_pc)//32,:]


        #remove points too close to origin
        d = np.sqrt(np.sum(nerf_pc**2, axis = 1))
        good  = np.argwhere(d > 1)[:,0]
        nerf_pc = nerf_pc[good]

        # #remove points out of range of sensor (only needed for Mai City)
        # d = np.sqrt(np.sum(nerf_pc**2, axis = 1))
        # good = np.argwhere(d < 100)[:,0]
        # nerf_pc = nerf_pc[good]

        nerf_pc = nerf_pc[spin_part*len(real_pc)//8:(spin_part+1)*len(nerf_pc)//8,:]
        real_pc = real_pc[spin_part*len(real_pc)//8:(spin_part+1)*len(real_pc)//8,:]

        points1 = torch.tensor(nerf_pc)
        points2 = torch.tensor(real_pc)

        dists = torch.cdist(points1, points2)
        # print(dists)
        # Find the minimum distances
        min_dists1 = dists.min(dim=1)[0]  # from points1 to points2
        min_dists2 = dists.min(dim=0)[0]  # from points2 to points1

    #     print(min_dists1, min_dists1.mean())
    #     print(min_dists2, min_dists2.mean())

        cd = (min_dists1.mean() + min_dists2.mean())/2        
        accuracy = min_dists2.mean() #TEST-- try flipping these values...
        completeness = min_dists1.mean()
        print("\n cd:", cd, "  accuracy: ", accuracy, " completeness:", completeness)

        cd_hist = np.append(cd_hist, cd)
        accuracy_hist = np.append(accuracy_hist, accuracy)
        completeness_hist = np.append(completeness_hist, completeness)
        
        good1 = np.where(min_dists1 < 0.1)[0]
        good1 = len(good1)/len(min_dists1)

        good2 = np.where(min_dists2 < 0.1)[0]
        good2 = len(good2)/len(min_dists2)

        good1_hist = np.append(good1_hist, good1)
        good2_hist = np.append(good2_hist, good2)
        print("good1, good2:", good1, good2)

In [ ]:
fig, ax = p.subplots()
# ax.plot(cd_hist, label = "CD")
# ax.plot(completeness_hist, label = "completeness")
# ax.plot(accuracy_hist, label = "accuracy")
ax.plot(good1_hist, label = "gh1")
ax.plot(good2_hist, label = "gh2")
ax.plot( np.mean(good1_hist**2)*np.ones([400]))

cutoff = 5000

print(np.mean(cd_hist[:cutoff]))
print(np.mean(accuracy_hist[:cutoff]))
print(np.mean(completeness_hist[:cutoff]))
print(np.mean((good1_hist)[:cutoff])) #too high to be F-score...?
print(np.mean((good2_hist)[:cutoff])) #too high to be F-score...?

In [ ]:
#DEBUG ONLY-- look at data distribution witout tails

bidx = np.linspace(0,3,100)

fig, ax = p.subplots()
d1 = min_dists1.numpy()
d1_good_only = d1[d1 < 0.2]

d2 = min_dists2.numpy()
d2_good_only = d2[d2 < 0.2]

ax.hist(min_dists1, bins = bidx, alpha = 0.5);
ax.hist(d1_good, bins = bidx, alpha = 0.5);

print(np.mean(d1))
print(np.mean(d1_good_only))

print(np.mean(d2))
print(np.mean(d2_good_only))

# TEST: Get F-Score using torch cdists

In [ ]:
points1 = torch.tensor(nerf_pc)
points2 = torch.tensor(real_pc)

dists = torch.cdist(points1, points2)
# print(dists)
# Find the minimum distances
min_dists1 = dists.min(dim=1)[0]  # from points1 to points2
min_dists2 = dists.min(dim=0)[0]  # from points2 to points1
print(min_dists1, min_dists1.mean())

good1 = np.where(min_dists1 < 0.1)[0]
print(len(good1)/len(min_dists1))

good2 = np.where(min_dists2 < 0.1)[0]
print(len(good2)/len(min_dists2))

In [ ]:
real_pc = torch.tensor(real_pc)
nerf_pc = torch.tensor(nerf_pc)

if len(real_pc.shape) == 2:
    real_pc = real_pc.unsqueeze(0)  # Add batch dimension
if len(nerf_pc.shape) == 2:
    nerf_pc = nerf_pc.unsqueeze(0)  # Add batch dimension
    
# Convert to float
real_pc = real_pc.float()
nerf_pc = nerf_pc.float()
print(nerf_pc.shape)

chamferDist = ChamferDistance()
dist_forward = chamferDist(nerf_pc, real_pc)
print(dist_forward)
# print(dist_forward/ nerf_pc.shape[1])
# dist_bidirectional = chamferDist(nerf_pc, real_pc, bidirectional=True)
# print(dist_bidirectional)
# print(dist_bidirectional/ nerf_pc.shape[1])

# #TEST -- modify chamfer.py to produce output for calculating F-scores
# # dist_forward = chamferDist(nerf_pc, real_pc, reduction = 'fscore')
# dist_forward = chamferDist(nerf_pc, real_pc, reduction = 'fscore', reverse = True)
# # dist_forward is actually a SQUARED distance, so I need to take the sqrt first
# print(dist_forward)
# dist_forward = torch.sqrt(dist_forward)
# print(dist_forward)

# bad = np.argwhere(dist_forward > 0.2)

# print(np.shape(dist_forward)[1])
# print(np.shape(bad)[1])

# print(1 - np.shape(bad)[1] / np.shape(dist_forward)[1])

In [ ]:
#calculate map accuracy
good = np.argwhere(dist_forward < 0.2)[1,:]
print(len(good))
print(len(dist_forward[0]))
a = torch.mean(dist_forward[0][good])
# a = torch.mean(dist_forward[0])
print(a)

# print(dist_forward[0,:])

In [ ]:
tp = 0.9341 #true positives
fp = 1 - 0.9341 #false positives

#false negatives -- no estimated point close enough
fn = 1-0.9286 #use reverse nn distnaces here


# precision
p = tp/(tp+fp)

#recall
r = tp/(tp+fn)

f_score = 2*(p*r)/(p+r)
print(f_score)


In [ ]:
# print(savepc)
# print(pc1_aligned)
# print(dist_forward/ np.shape(nerf_pc)[1])

from vedo import *
from ipyvtklink.viewer import ViewInteractiveWidget

plt = Plotter(N = 1, axes = 0, bg = (1, 1, 1), interactive = True) #axes = 4 (simple), 1(scale)
disp=[]

#crop real PC down the same as NeRF output so bidirectional CD doesn't punish non-overlaps
# test = real_pc[0,len(real_pc[0,:,:])//8:7*len(real_pc[0,:,:])//8,:]
# print(np.shape(test))
# print(len(real_pc[0,:,:])//8)
# disp.append(Points(test, c = "red", r = 5))

disp.append(Points(real_pc[0,:,:], c = "red", r = 5))
disp.append(Points(nerf_pc[0,:,:], c = "blue", r = 5))

plt.show(disp, "testing CD")
ViewInteractiveWidget(plt.window)


# calculate chamfer distance between NeRF output true distance for 25 frames Mai City Dataset

In [ ]:
CD_hist = []
chamferDist = ChamferDistance()

for frame_idx in range(25):
    print(frame_idx)
    nerf_pc = np.load("generatedPointClouds/MaiCityNeRFOutput_frame_" + str(frame_idx) + ".npy")
    real_pc = np.load("generatedPointClouds/MaiCityReal_frame_" + str(frame_idx) + ".npy")

    #remove points too close to origin
    d = np.sqrt(np.sum(nerf_pc**2, axis = 1))
    good  = np.argwhere(d > 1)[:,0]
    nerf_pc = nerf_pc[good]

    #remove points out of range of sensor (only needed for Mai City)
    d = np.sqrt(np.sum(nerf_pc**2, axis = 1))
    good = np.argwhere(d < 100)[:,0]
    nerf_pc = nerf_pc[good]
    
#     print(np.shape(nerf_pc))
#     nerf_pc = nerf_pc[::3,:]
#     real_pc = real_pc[::3,:]
#     print(np.shape(nerf_pc))

    real_pc = torch.tensor(real_pc)
    nerf_pc = torch.tensor(nerf_pc)

    if len(real_pc.shape) == 2:
        real_pc = real_pc.unsqueeze(0)  # Add batch dimension
    if len(nerf_pc.shape) == 2:
        nerf_pc = nerf_pc.unsqueeze(0)  # Add batch dimension

    # Convert to float
    real_pc = real_pc.float()
    nerf_pc = nerf_pc.float()
    
#     dist_bidirectional = chamferDist(nerf_pc, real_pc, bidirectional=True)
#     dist_bidirectional /= nerf_pc.shape[1]
#     print(dist_bidirectional)
#     CD_hist = np.append(CD_hist, dist_bidirectional)


    dist_forward = chamferDist(nerf_pc, real_pc)
#     dist_forward /= nerf_pc.shape[1]
    print(dist_forward)
    CD_hist = np.append(CD_hist, dist_forward)

In [ ]:
from matplotlib import pyplot as p 

print(np.mean(CD_hist))
# np.save("generatedPointClouds/MaiCity_CD_hist.npy", CD_hist)
# CD_hist = np.load("generatedPointClouds/MaiCity_CD_hist.npy")
# print(CD_hist)
fig, ax = p.subplots()
ax.plot(CD_hist)

In [ ]:
from matplotlib import pyplot as p

# CD_hist_2 = CD_hist.copy()

fig, ax = p.subplots()
ax.set_title("Chamfer Distance for Mai City Dataset")
# ax.plot(CD_hist_15, label = "sample every 15th")
# ax.plot(CD_hist_10, label = "sample every 10th")
# ax.plot(CD_hist_5, label = "sample every 5th")
# ax.plot(CD_hist_2, label = "sample every 2nd")
ax.plot(CD_hist, label = "CD")
ax.set_xlabel("test frame index")
ax.set_ylabel("Chamfer Distance (cm)")
ax.legend(loc="best")

In [ ]:
#combine all NeRF outputs to single point cloud to render with open3d
combined_pc = np.zeros([0,3])
for frame_idx in range(50): #25
    print(frame_idx)
#     nerf_pc = np.load("generatedPointClouds/MaiCityNeRFOutput_frame_" + str(frame_idx) + ".npy") #
    nerf_pc = np.load("generatedPointClouds/MaiCityNeRFOutput_velodyne_format_frame_" + str(frame_idx) + ".npy") #OS1 projections
#     nerf_pc = np.load("generatedPointClouds/MaiCityReal_frame_" + str(frame_idx) + ".npy") #DEBUG -- true scans

    #remove near zero points
#     nerf_pc = nerf_pc*200
    dists = np.sqrt(nerf_pc[:,0]**2 + nerf_pc[:,1]**2 + nerf_pc[:,2]**2)
    good = np.argwhere(dists > 2)[:,0]
#     print(good)
    nerf_pc = nerf_pc[good]

    nerf_pc = nerf_pc + np.array([frame_idx*.5,0.,0.])
#     nerf_pc = nerf_pc + np.array([frame_idx*1.,0.,0.])
    
    combined_pc = np.append(combined_pc, nerf_pc, axis = 0)

# print(np.shape(combined_pc))

# #crop to help with marching cubes??
# combined_pc = combined_pc[abs(combined_pc[:,0]) < 20]
# combined_pc = combined_pc[abs(combined_pc[:,1]) < 20]
# np.save("generatedPointClouds/CombinedMaiCity.npy", combined_pc)

import open3d as o3d
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(combined_pc)
o3d.io.write_point_cloud("MaiCityNeRF.ply", pcd)

In [ ]:
#TEST -- load and crop HD Map to test marching cubes
import trimesh
pl = '/media/derm/06EF-127D4/Newer College Dataset/new-college-29-01-2020-1cm-resolution-1stSection - mesh.ply'
raw_points = trimesh.load(pl).vertices
# points[:,2] = -points[:,2] #flip z
# points = points[::10,:] #downsample
points = raw_points.copy()

points = points[points[:,0]>0]
points = points[points[:,1]> - 2]

combined_pc = points
# np.save("generatedPointClouds/NCTestMap.npy", combined_pc)

import open3d as o3d
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(combined_pc)
o3d.io.write_point_cloud("NCTestMap.ply", pcd)


In [ ]:
plt = Plotter(N = 1, axes = 0, bg = (1, 1, 1), interactive = True) #axes = 4 (simple), 1(scale)
disp=[]

disp.append(Points(combined_pc, c = "red", r = 2.5, alpha = 0.3))
disp.append(Points(raw_points, c = "black", r = 2.5, alpha = 0.1))

plt.show(disp, "testing CD")
ViewInteractiveWidget(plt.window)


### Load 100 raw frames from Mai City sequence 01 and combine to a single point cloud

In [ ]:
# test = np.loadtxt("/media/derm/06EF-127D4/mai_city/txt/01/000000")

combined_pc = np.zeros([0,3])
for idx1 in range(100):
    print(idx1)
    pcfn = "/media/derm/06EF-127D4/mai_city/txt/01/" +  f"{idx1:06}"
    
    pc = np.loadtxt(pcfn)
    pc += np.array([1.*idx1, 0.,0.])
    pc = pc[::2]
    
    combined_pc = np.append(combined_pc, pc, axis = 0)

print(combined_pc)
import open3d as o3d
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(combined_pc)
o3d.io.write_point_cloud("MaiCityGTMap.ply", pcd)

# Calculate CD using pytorch3d chafer_distance() loss

In [ ]:
import pytorch3d

In [ ]:
%%bash
pip show torch